## Expert Knowledge Worker

### A question answering agent that is an expert knowledge worker
### To be used by employees of Insurellm, an Insurance Tech company
### The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

In [2]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [4]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [5]:
MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [6]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "your-key-if-not-using-env")

In [10]:
folders = glob.glob(
    "/Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_AI/dev_proj_AI/practise_ai_udemy_llm_engineering/ref_udemy_llm_engineering/week5/knowledge-base/*"
)

text_loader_kwargs = {"encoding": "utf-8"}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    # define loader for md docs
    loader = DirectoryLoader(
        folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs
    )
    # initiate loader
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [11]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1088, which is longer than the specified 1000


In [12]:
len(chunks)

123

In [14]:
doc_types = set(chunk.metadata["doc_type"] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: company, products, employees, contracts


## A sidenote on Embeddings, and "Auto-Encoding LLMs"

We will be mapping each chunk of text into a Vector that represents the meaning of the text, known as an embedding.

OpenAI offers a model to do this, which we will use by calling their API with some LangChain code.

This model is an example of an "Auto-Encoding LLM" which generates an output given a complete input.
It's different to all the other LLMs we've discussed today, which are known as "Auto-Regressive LLMs", and generate future tokens based only on past context.

Another example of an Auto-Encoding LLMs is BERT from Google. In addition to embedding, Auto-encoding LLMs are often used for classification.

### Sidenote

In week 8 we will return to RAG and vector embeddings, and we will use an open-source vector encoder so that the data never leaves our computer - that's an important consideration when building enterprise systems and the data needs to remain internal.

In [ ]:
# # hugging face embeddings

# from langchain.embeddings import HuggingFaceEmbeddings

# embeddings_hf = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/all-MiniLM-L6-v2"
# )

# # delete vector store if exists
# if os.path.exists(db_name):
#     Chroma(
#         persist_directory=db_name, embedding_function=embeddings_hf
#     ).delete_collection()

# # create a vectorstore
# vectorstore = Chroma.from_documents(
#     documents=chunks, embedding=embeddings_hf, persist_directory=db_name
# )

# vectorstore._collection.count()

123

In [29]:
# collection = vectorstore._collection
# sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
# dimensions = len(sample_embedding)
# dimensions

384

In [38]:
# openai embeddings
embeddings_oai = OpenAIEmbeddings()

# delete vector store if exists
if os.path.exists(db_name):
    Chroma(
        persist_directory=db_name, embedding_function=embeddings_oai
    ).delete_collection()

# create a vectorstore
vectorstore = Chroma.from_documents(
    documents=chunks, embedding=embeddings_oai, persist_directory=db_name
)

vectorstore._collection.count()

123

In [39]:
collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
dimensions

1536

## Visualizing the Vector Store

Let's take a minute to look at the documents and their embedding vectors to see what's going on.

In [40]:
# Prework

result = collection.get(include=["embeddings", "documents", "metadatas"])
vectors = np.array(result["embeddings"])
documents = result["documents"]
doc_types = [metadata["doc_type"] for metadata in result["metadatas"]]
colors = [
    ["blue", "green", "red", "orange"][
        ["products", "employees", "contracts", "company"].index(t)
    ]
    for t in doc_types
]

In [41]:
# We humans find it easier to visalize things in 2D!
# Reduce the dimensionality of the vectors to 2D using t-SNE
# (t-distributed stochastic neighbor embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(
    data=[
        go.Scatter(
            x=reduced_vectors[:, 0],
            y=reduced_vectors[:, 1],
            mode="markers",
            marker=dict(size=5, color=colors, opacity=0.8),
            text=[
                f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)
            ],
            hoverinfo="text",
        )
    ]
)

fig.update_layout(
    title="2D Chroma Vector Store Visualization",
    scene=dict(xaxis_title="x", yaxis_title="y"),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40),
)

fig.show()

In [42]:
# Let's try 3D!

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=reduced_vectors[:, 0],
            y=reduced_vectors[:, 1],
            z=reduced_vectors[:, 2],
            mode="markers",
            marker=dict(size=5, color=colors, opacity=0.8),
            text=[
                f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)
            ],
            hoverinfo="text",
        )
    ]
)

fig.update_layout(
    title="3D Chroma Vector Store Visualization",
    scene=dict(xaxis_title="x", yaxis_title="y", zaxis_title="z"),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40),
)

fig.show()

# Time to use LangChain to bring it all together

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">PLEASE READ ME! Ignoring the Deprecation Warning</h2>
            <span style="color:#900;">When you run the next cell, you will get a LangChainDeprecationWarning 
            about the simple way we use LangChain memory. They ask us to migrate to their new approach for memory. 
            I feel quite conflicted about this. The new approach involves moving to LangGraph and getting deep into their ecosystem.
            There's a fair amount of learning and coding in LangGraph, frankly without much benefit in our case.<br/><br/>
            I'm going to think about whether/how to incorporate it in the course, but for now please ignore the Depreciation Warning and
            use the code as is; LangChain are not expected to remove ConversationBufferMemory any time soon.
            </span>
        </td>
    </tr>
</table>

In [43]:
# setup llm
llm = ChatOpenAI(model=MODEL, temperature=0.7)

# setup conversation memory for chat
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# setup the retriver to get the documents for context
retriever = vectorstore.as_retriever()

# combine all these for chatbot
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, retriever=retriever, memory=memory
)

In [44]:
query = "Can you describe Insurellm in a few sentences"
result = conversation_chain.invoke({"question": query})
print(result["answer"])

Insurellm is an innovative insurance tech firm founded in 2015 by Avery Lancaster, designed to disrupt the insurance industry with cutting-edge products. The company offers four main software products: Carllm for auto insurance companies, Homellm for home insurance companies, Rellm for the reinsurance sector, and Marketllm, a marketplace connecting consumers with insurance providers. With a workforce of 200 employees and over 300 clients worldwide, Insurellm is committed to transforming the insurance landscape through technology and innovation.


In [45]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, retriever=retriever, memory=memory
)

## Now we will bring this up in Gradio using the Chat interface -

A quick and easy way to prototype a chat with an LLM

In [46]:
def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [47]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
